In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[261]:


import numpy as np
import matplotlib.pyplot as plt
import math
import random
import h5py
import copy
from scipy.optimize import curve_fit


# In[262]:


class Sig:
    def __init__(self):
        self.number_groups = 0
        self.sig_f = []
        self.capture = []
        self.scattering = []
        self.total = []
        self.number_of_production_neutrons = []
        self.c_value = []
        self.virtual = []
        
    def __init__(self, number_groups, sig_f, sig_c, sig_scattering_matrix, number_of_production_neutrons,
                 cs_xi_table, sig_t):
        self.number_groups = number_groups
        self.sig_f = sig_f
        self.sig_c = sig_c
        self.sig_s = []
        self.sig_scattering_matrix = sig_scattering_matrix

        for i in range(0, len(self.sig_scattering_matrix)):
            self.sig_s.append(sum(self.sig_scattering_matrix[i]))
            
        self.number_of_production_neutrons = number_of_production_neutrons
        self.cs_xi_table = cs_xi_table
        self.sig_t = sig_t
        self.delta_xs = max(sig_t)/2.
        max_total_cs = max(self.sig_t)
        self.virtual = [self.delta_xs]
        
               
    def get_virtual_cs(self):
        max_total_cs = max(self.sig_t)
        self.virtual = [cross_section - max_total_cs + self.delta_xs for cross_section in self.sig_t]
        
    def return_virtual(self):
        return self.virtual
        
    def get_fission_probability(self, energy_group_idx):
        return self.sig_f[energy_group_idx] / self.sig_t[energy_group_idx]
    
    def get_capture_probability(self, energy_group_idx):
        return self.sig_c[energy_group_idx] / self.sig_t[energy_group_idx]
        
    def get_scatter_probability(self, energy_group_idx):
        return self.sig_s[energy_group_idx] / self.sig_t[energy_group_idx]
    
    def get_number_of_groups(self):
        
        return self.number_groups
        


# In[263]:


energy_groups_2 = [0, 6.25, 2 * 1E10]

class Positon:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
class Direction:
    def __init__(self, ets, phi):
        self.tetta_x = math.sin(ets) * math.cos(phi)
        self.tetta_y = math.sin(ets) * math.sin(phi)
        self.tetta_z = math.cos(ets)  


# In[264]:


class Particle:
    def __init__(self):
        self.coordinates = Positon(0, 0, 0)
        self.direction = Direction(0, 0)
        self.energy = 0.
        self.weight = 1
        self.energy_groups = []
        self.terminated = False
        self.path = 0.
        self.last_cell = 0
        self.energy_group = 0
        
    def set_coordinates(self, x, y, z):
        self.coordinates = Positon(x, y, z)
        
    def set_direction(self, ets, phi):
        self.direction = Direction(ets, phi)
    
    def set_direction_angle(self, angle):
        self.direction = angle
        
    def set_energy_groups(self, energy_groups):
        self.energy_groups = energy_groups
        
    def get_energy_group(self): 
        
        return self.energy_group
     #   res = next(x for x, val in enumerate(self.energy_groups)
      #                            if val > self.energy)        
       # return res - 1
    
    def set_energy_group(self, energy_group):
        
        self.energy_group = energy_group       
    
    def set_terminated(self):
        self.terminated = True
        
    def is_terminated(self):
        return self.terminated
        
    def get_weight(self):
        return self.weight
    
    def set_weight(self, weight):
        self.weight = weight
    
    def set_particle_deleted(self):
        self.weight = self.weight * 0
        
    def set_particle_fission(self, additional_weight):
        self.weight = self.weight * additional_weight
        
    def set_multiplicity(self, additional_weight):
        self.weight = self.weight * additional_weight
              
    def is_particle_deleted(self):
        return self.weight < 0.0001
    
    def add_path(self, path):
        self.path += path
    
    def get_path(self):
        return self.path
    
    def set_path(self, path):
        self.path = path
        
    def print_direction(self):
        print(str(self.direction.tetta_x) + " " + str(self.direction.tetta_y) + " " + str(self.direction.tetta_z))
        
    def print_coordinates(self):
        print(str(self.coordinates.x) + "  " + str(self.coordinates.y) + " " + str(self.coordinates.z))  
        


# In[265]:


class Plane:
    def __init__(self, A, B, C, D):
        self.A = A
        self.B = B
        self.C = C
        self.D = D
        
    def distance(self, particle):
        vp = (self.A * particle.direction.tetta_x + self.B * particle.direction.tetta_y + 
              self.C * particle.direction.tetta_z)
        
        
        if (abs(vp) < 1e-9):
            return -1
        
        distance = (self.D - self.A * particle.coordinates.x - self.B * particle.coordinates.y 
                    - self.C * particle.coordinates.z) / vp
        
        return distance 
    
    def get_normal(self):
        sq = math.sqrt(self.A * self.A + self.B * self.B + self.C * self.C)
        a_n = self.A / sq
        b_n = self.B / sq
        c_n = self.C / sq
        return [a_n, b_n, c_n]
    

    def get_sign(self, particle):
      
        sign = (self.A * particle.coordinates.x + self.B * particle.coordinates.y +
        self.C * particle.coordinates.z - self.D)
        
        if sign == 0:
            return sign    
            
        if sign < 0:
            return -1
        else:
            return 1
        
    def get_xml(self):
        
        obj_xml = " Plane with A "+ str(self.A) + " B "+ str(self.B) + " C "+ str(self.C) + " D "+ str(self.D)
        
        return obj_xml
    


# In[266]:


class Universe:
    def __init__(self, cells, xs):
        self.cells = cells
        self.xs = xs
        self.delta_xs = 0.
        
    def calculate_delta_xs(self):
        
        delta_xs = []
        for i in range(0, self.xs[0].number_groups):
            energy_group_maximum = -1
            for xs in self.xs:
                energy_group_maximum = max(energy_group_maximum, xs.sig_t[i])
            delta_xs.append(energy_group_maximum)
            
        self.delta_xs = delta_xs                          
            
    def get_delta_xs(self, energy_group):
        
        return self.delta_xs[energy_group]
    
    def get_xs_by_coordinates(self, particle):
        
        for i in range(0, len(cells)):
            cell = cells[i]
            if cell.is_inside(particle):
                particle.last_cell = i
                return self.xs[i]
        return []
    
    def is_inside(self, particle):
        
        for cell in self.cells:
            if cell.is_inside(particle):
                return True
        return False
        
    
    def is_boudary_reflective(self, particle):
        
        minumum = np.inf
        idx_of_min = 0.
        for i in range(0, len(self.cells)):
            distance, surface_idx = self.cells[i].get_minimum_distance(particle)
            if abs(distance) < minumum:
                minumum = distance
                idx_of_min = i
        if self.cells[idx_of_min].is_boudary_reflective(particle):
            return True
        else:
            return False
        
    def reflect_particle(self,  particle):
        self.cells[particle.last_cell].reflect_particle(particle)       


# In[267]:


def set_random_direction(particle):
    r = 1.0
    r1 = 1.0
    while (r**2 + r1**2 > 1.0):
        r = 2. * random.uniform(0, 1) - 1.
        r1 = 2. * random.uniform(0, 1) - 1.
        
    direction_x = 2.0 * r**2 + 2. * r1**2 - 1.0  
    particle.direction.tetta_x = direction_x
    particle.direction.tetta_y = r * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))
    particle.direction.tetta_z = r1 * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))


# In[268]:


import random

def make_initial_sources(number_of_paricles, box_size, zero_point, energy=10.0e6):
    
    step_x = box_size[0]/number_of_paricles
    step_y = box_size[1]/number_of_paricles
    step_z = box_size[2]/number_of_paricles
    
    x_coord = zero_point[0]
    y_coord = zero_point[1]
    z_coord = zero_point[2]
    
    sources = []
    for i in range(0, number_of_paricles):
        for j in range(0, number_of_paricles):
            for k in range(0, number_of_paricles):
                current_particle = Particle()
                current_particle.set_coordinates(x_coord + k * step_x, y_coord + j * step_y, z_coord+ i * step_z)
                set_random_direction(current_particle)
                energy_group = random.choice([0, 1])
                current_particle.set_energy_group(energy_group)
                current_particle.energy = energy
                current_particle.set_energy_groups(energy_groups_2)
                sources.append(current_particle)

    return sources


# In[269]:


def get_free_path(particle, delta_xs):
    
    energy_group_idx = particle.get_energy_group()
    sig_t = delta_xs
    free_path = -math.log(random.uniform(0, 1)) / sig_t
    
    return free_path  


# In[ ]:





# In[270]:


class Cell:
    def __init__(self):
        self.surfaces = []
        self.boundaries_type = []
        self.signs = []
        self.size = 0
        
    def set_boundaries_type(self, boundaries_type):
        self.boundaries_type = boundaries_type        
        
    def set_box_sizes(self, xm_size, ym_size, zm_size, x_size, y_size, z_size):
        self.xm_size = xm_size
        self.ym_size = ym_size
        self.zm_size = zm_size
        
        self.x_size = x_size
        self.y_size = y_size
        self.z_size = z_size
        
    def set_zero_point(self, x_0, y_0, z_0):
        self.x_0 = x_0
        self.y_0 = y_0
        self.z_0 = z_0
        
    def __init__(self, surfaces, signs):
        self.surfaces = surfaces
        self.signs = signs
        self.size = len(self.signs)
           
    def is_inside(self, particle):
        
        is_inside = True
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                is_inside = False
                
        return is_inside
    
    def is_boudary_reflective(self, particle):
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                if self.boundaries_type[i] == 'reflective':
                    return True
                else:
                    return False
        
    
    def get_minimum_distance(self, particle):
        
        minimum_distance = math.inf
        surface_idx = 0
        for i in range(0, len(self.surfaces)):
            current_distance = abs(self.surfaces[i].distance(particle))
            if current_distance > 0:
                minimum_distance = min(minimum_distance, current_distance)
                surface_idx = i
        return minimum_distance, surface_idx
    
    def get_xml(self):
        
        xml_obj = []
        for i in range(0, self.size):
            xml_obj.append(surfaces[i].get_xml())
            
        return xml_obj
    
    def reflect_particle(self, particle):
        
        if particle.coordinates.x > (self.x_size + self.x_0):
            delta = particle.coordinates.x % self.x_size
            particle.coordinates.x = self.xm_size + math.fabs(delta)
        
        if particle.coordinates.y > (self.y_size + self.y_0):
            
            delta = particle.coordinates.y % self.y_size
            particle.coordinates.y = self.ym_size + math.fabs(delta)
            
            
        if particle.coordinates.z > (self.z_size + self.z_0):
            
            delta = particle.coordinates.z % self.z_size
            particle.coordinates.z = self.zm_size + math.fabs(delta)
        
        
        if particle.coordinates.y < (self.ym_size + self.y_0):
            delta = particle.coordinates.y % self.ym_size
            particle.coordinates.y = self.y_size - math.fabs(delta)
            
            
        if particle.coordinates.x < (self.xm_size + self.x_0):
            delta = particle.coordinates.x % self.xm_size
            particle.coordinates.x = self.x_size - math.fabs(delta)
            
        if particle.coordinates.z < (self.zm_size + self.z_0):
            delta = particle.coordinates.z % self.zm_size
            particle.coordinates.z = self.z_size - math.fabs(delta)
                    


# In[271]:


class Multigroup_estimator:
    def __init__(self):
        self.groups_before = []
        self.weights_before = []
        self.groups_after = []
        self.weights_after = []
        
    def get_flux_ration_n_m(self, first, secound):
        
        number_before = 0.
        number_after = 0.
        for i in range(0, len(self.groups_before)):
            
            if self.groups_before[i] == first:
                number_before += self.weights_before[i]           
                if self.groups_after[i] == secound:
                    number_after += self.weights_after[i]
        
        flux_ratio = number_after / number_before
    
        return flux_ratio   


# In[272]:


def move_particle(particle, t):
    new_x = particle.direction.tetta_x * t + particle.coordinates.x
    new_y = particle.direction.tetta_y * t + particle.coordinates.y
    new_z = particle.direction.tetta_z * t + particle.coordinates.z
    particle.set_coordinates(new_x, new_y, new_z)


# In[273]:


def is_collision_virual(particle, c_s, delta_xs):
    
    energy_group_idx = particle.get_energy_group()
    random_number = random.uniform(0, 1)
    
    virtual_cs = delta_xs
    
    total_cs = c_s.sig_t[energy_group_idx]
    
    if total_cs / virtual_cs < random_number:
        return True
    else:
        return False


# In[274]:


def process_virtual_collision(particle, free_path):
    particle.add_path(free_path)


# In[275]:


def process_scattering(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()   
    
    energy_group_probability = []
    energy_scattering_matrix = c_s.sig_scattering_matrix[energy_group_idx]
    
    majorant = sum(energy_scattering_matrix)
    for i in range(0, len(energy_scattering_matrix)):
        energy_group_probability.append(energy_scattering_matrix[i] / majorant)
        
    energy_groups_idxes = list(range(0, len(energy_scattering_matrix)))
    energy_group_idx = np.random.choice(energy_groups_idxes,  p=energy_group_probability)
    
    particle.set_energy_group(energy_group_idx)     


# In[276]:


def process_fission(particle, c_s):
    energy_group_idx = particle.get_energy_group()
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]
    particle.set_particle_fission(number_of_production_neutrons)
    particle.set_terminated()
    energy_groups_idxes = list(range(0, c_s.number_groups))
    xi_matrix = c_s.cs_xi_table
    energy_group_idx = np.random.choice(energy_groups_idxes,  p=xi_matrix)
    particle.set_energy_group(energy_group_idx)
    


# In[277]:


def process_real_collision(particle, free_path, c_s, flux_group_estimator):
    
    energy_group_idx = particle.get_energy_group()
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]   
    capture_probability = c_s.get_capture_probability(energy_group_idx)
    scatter_probability = c_s.get_scatter_probability(energy_group_idx)  
    fission_probability = c_s.get_fission_probability(energy_group_idx)  
    
    type_collision = np.random.choice(['capture', 'scatter', 'fission'], p=[capture_probability, scatter_probability,
                                                                            fission_probability])
    if type_collision == 'capture':
        particle.set_terminated()
        particle.set_weight(0.)
        
    if type_collision == 'scatter':
        process_scattering(particle, c_s)

        
    if type_collision == 'fission':    
        process_fission(particle, c_s)
        


# In[278]:


def delete_absorpbed_paricles(particles):
    
    existing_particles = []
    for i in range(0, len(particles)):
        particle = particles[i]
        if particle.get_weight() > 0.000001:
            existing_particles.append(particle)
                     
    return existing_particles


# In[279]:


0.0432 / 0.2208


# In[280]:


0.020432/(0.020432 + 0.247516)


# In[281]:


#      self.groups_before = []
 #     self.weights_before = []
  #    self.groups_after = []
   #   self.weights_after = []
      


# In[282]:


def process_one_particle_history(particle, universe, estimators, flux_estimator):
    
    sum_collisions = 0.
    universe.calculate_delta_xs()
    
    while not particle.is_terminated():
        
        energy_group_before = particle.get_energy_group()
        weight_before = particle.get_weight()
        

        current_xs = universe.get_xs_by_coordinates(particle)
        energy_group_idx = particle.get_energy_group()
        delta_xs = universe.get_delta_xs(energy_group_idx)
        free_path = get_free_path(particle, delta_xs)        
        move_particle(particle, free_path)
        current_xs = universe.get_xs_by_coordinates(particle)
        
        flux_estimator.groups_before.append(particle.get_energy_group())
        flux_estimator.weights_before.append(particle.get_weight())
        
        if not universe.is_inside(particle):
            if universe.is_boudary_reflective(particle):
                universe.reflect_particle(particle)
                current_xs = universe.get_xs_by_coordinates(particle)

            else:
                particle.set_terminated()
                particle.set_weight(0.)
                return particle, sum_collisions 
        for k in estimators:
            k.add_collision(particle)
        
        if is_collision_virual(particle, current_xs, delta_xs):
            process_virtual_collision(particle, free_path)
        else:
            process_real_collision(particle, free_path, current_xs, flux_estimator)
            
    
    return particle, sum_collisions


# In[283]:


def get_weights(particles):
    
    weights = []
    for i in range(0, len(particles)):
        weights.append(particles[i].weight)
        
    return weights


# In[284]:


def make_sources(particles):
    for i in range(0, len(particles)):
        particles[i].terminated = False
        set_random_direction(particles[i])       


# In[285]:


def splitting_secound_version(particles):
    
    initial_size = len(particles)
    
    for i in range(0, initial_size):
        particle = particles[i]
        current_weight = particle.get_weight()
        if current_weight > 1:
            n_value = math.floor(current_weight)
            random_value = random.uniform(0, 1)
            if current_weight - n_value >= random_value:
                n_value += 1
            if n_value > 1:
                particle.set_weight(current_weight / n_value)
                for j in range(0, n_value - 1):
                    particles.append(copy.deepcopy(particle))


# In[286]:


def normalise_weights(particles, batch_size):

    sum_weights = 0.
    
    for i in range(0, len(particles)):
        sum_weights += particles[i].get_weight()

    for i in range(0, len(particles)):
        particles[i].weight = (particles[i].weight * batch_size) / sum_weights   
        
    return particles


# In[287]:


def russian_roulette(weights_previous, particles_current):
    
    for i in range(0, len(particles_current)):
        particle = particles_current[i]
        if particle.weight > 0 and particle.weight < 0.5 and particle.weight < weights_previous[i]:
            probability_terminate = 1. - particle.weight / weights_previous[i]
            random_number = random.uniform(0, 1)
            if probability_terminate >= random_number:
                particle.set_weight(0.)
            else:
                particle.set_weight(weights_previous[i])


# In[288]:


def terminate_outside_particles(batch_particles, universe):
    
    for j in range(0, len(batch_particles)):
        particle = batch_particles[j]
        if not universe.is_inside(particle):
            print("ERROR! outside particle")


# In[289]:


def reset_estimators(estimators, weight_previos, volume, sum_collisions, c_s):
    
    sig_t = c_s.sig_t[0]

    for estimator in estimators:
        estimator.collision_sum[-1] = estimator.collision_sum[-1]
        estimator.collision_sum.append(0.)  


# In[290]:


def get_std(values):
    
    current_std = np.std(values)/np.sqrt((len(values) - 1.))
  
    return current_std   


# In[291]:


def calculate_k_effective(idx, weights, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, std_k_effective):
    
    keff_cycle = sum(weights) / initial_size
    if idx > number_inactive:
        k_effective.append(keff_cycle)
        
    if idx > number_inactive + 1:
    
        k_effective_exp.append(sum(k_effective) / len(k_effective))
        
        std_k_effective_current = get_std(k_effective)
            
        std_k_effective.append(std_k_effective_current)

        print(" keff_cycle , k_effective_exp, std_k_effective " + str(keff_cycle) + "   " + str(k_effective_exp[-1]) +
                                    "  "+ str(std_k_effective[-1]))


# In[296]:


class Flux_group_estimator:
    def __init__(self, group):
        self.group = group
        self.collision_sum = [0]
        
    def add_collision(self, particle):
        if particle.get_energy_group() == self.group:
            self.collision_sum[-1]  += particle.get_weight()    


# In[297]:


import statistics
import time


def simulation_black_boundaries(universe, number_interations, number_inactive, number_of_particles, c_s, estimators,
                                volume=1):
    
    random.seed(time.time())
    k_effective = []
    k_effective_std = []
    k_effective_exp = []
    flux = []
    flux_exp = []
    num1 = random.randint(0,9)
    flux_group_estimators = []
    print(" num1  " + str(num1))
    
    box_size = [1., 1., 1.]
    zero_point = [0.1, 0.1, 0.1]
    initial_sources = make_initial_sources(number_of_particles, box_size, zero_point, energy=10.0e6)
    initial_size = len(initial_sources)
    
    weights_previous = [1.] * len(initial_sources)
    number_of_groups = universe.xs[0].get_number_of_groups()
    for i in range(0, number_interations):
        print("i == " + str(i))
        make_sources(initial_sources)
        batch_size = len(initial_sources)
        
        batch_particles = []
        flux_group_estimator = Multigroup_estimator()
        for j in range(0, batch_size):
            particle = initial_sources[j]           
            terminate_particle, sum_collisions = process_one_particle_history(particle, universe, 
                                                                            estimators, flux_group_estimator)
            batch_particles.append(terminate_particle)
            
            
        
        russian_roulette(weights_previous, batch_particles)
        
        batch_particles = delete_absorpbed_paricles(batch_particles)
        
        splitting_secound_version(batch_particles)
        
        weights_cycle = get_weights(batch_particles)
        
        calculate_k_effective(i, weights_cycle, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, k_effective_std)
        
        flux_group_estimators.append(flux_group_estimator)
        reset_estimators(estimators, weights_previous, volume, sum_collisions, c_s)              

        batch_particles = normalise_weights(batch_particles, initial_size)  

        initial_sources = batch_particles
        weights_previous = get_weights(batch_particles)
     
    return k_effective, k_effective_exp, k_effective_std, estimators, flux_group_estimators


# In[298]:


cs_fission_pu_239 = [0.08544, 0.0936]

cs_capture_pu_239 = [0.0144, 0.00480]

cs_scattering_pu_239 = [[0.23616, 0.0], [0.0432, 0.0792]]

cs_total_pu_239 = [0.3360, 0.2208]

cs_production_neutrons_pu_239 = [2.93, 3.10]

cs_xi_table_pu_239 = [0.425, 0.575]


pu_239 = Sig(2, cs_fission_pu_239, cs_capture_pu_239, cs_scattering_pu_239, 
                       cs_production_neutrons_pu_239,  cs_xi_table_pu_239, cs_total_pu_239)


energy_groups_2 = [0, 2 * 1E10]


# In[299]:


x_5 = Plane(1, 0, 0, 1000)
x_m5 = Plane(1, 0, 0, -1000)

y_5 = Plane(0, 1, 0, 1000)

y_m5 = Plane(0, 1, 0, -1000)


z_5 = Plane(0, 0, 1, 1000)
z_m5 = Plane(0, 0, 1, -1000)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

signs = [-1, +1, -1, +1, -1, +1]
slab = Cell(surfaces, signs)

boundaries_type = ["reflective", "reflective", "reflective", "reflective", "reflective", "reflective"]


slab.set_boundaries_type(boundaries_type)
slab.set_box_sizes(-1000, -1000., -1000, 1000, 1000., 1000 )
slab.set_zero_point(0., 0., 0.)


# In[300]:


cells = [slab]


# In[301]:


estimators = []


# In[302]:


materials = [pu_239]


# In[303]:


U235_Reactor = Universe(cells, materials)


# In[304]:


zero_group_estimator = Flux_group_estimator(0)
first_group_estimator = Flux_group_estimator(1)


# In[305]:


estimators = [zero_group_estimator, first_group_estimator]


# In[306]:


test_number_of_particles = 10
test_number_interations = 500
test_number_inactive = 50


# In[307]:


k_effective,k_effective_exp, k_effective_std, estimators_result, flux_group_result = simulation_black_boundaries(U235_Reactor,  
                           test_number_interations,  test_number_inactive, 
                          test_number_of_particles, pu_239, estimators)


# In[308]:


first_group_flux = []
secound_group_flux = []

number_particles = 1000
cs_total_pu_239 = [0.3360, 0.2208]

for i in range(0, len(estimators_result[0].collision_sum) - 1):
    first_group_flux.append(estimators_result[0].collision_sum[i]/(cs_total_pu_239[0]))

for i in range(0, len(estimators_result[1].collision_sum) - 1):
    secound_group_flux.append(estimators_result[1].collision_sum[i]/(cs_total_pu_239[1]))
    
average_flux_difference = []    
for i in range(0, len(first_group_flux)):
    average_flux_difference.append(secound_group_flux[i]/ first_group_flux[i])

std_flux = get_std(average_flux_difference)
flux_difference = sum(average_flux_difference)/len(average_flux_difference)


# In[309]:


difference = (flux_difference  - 0.675229) * 100000

print('difference flux == '+ str(difference)+ 'with std   '+ str(std_flux * 100000))


# In[310]:


difference = (2.683767  - k_effective_exp[-1]) * 100000

standart_deviation = k_effective_std[-1] * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(standart_deviation))


# In[311]:


def save_simulation_results_to_file(k_effective, file_name):
    
    f = open(file_name, "w")
    
    for i in range(0, len(k_effective)):
        f.write(" cycle k effective "+ str(k_effective[i]) + '\n')
            
    
    f.close()


# In[312]:


file_name = "output.txt"
save_simulation_results_to_file(k_effective, file_name)


# In[313]:


def save_flux_to_file(estimators, file_name):
        
    k = 0
    for estimator in estimators:
        
        result_file_name = file_name + str(k)+".txt"
        
        f = open(result_file_name, "w")
        array_without_zeros = estimator.collision_sum
        for i in range(0, len(array_without_zeros)):
            f.write(" flux "+ str(array_without_zeros[i]) +'\n')
        k += 1
        
        f.close()


# In[314]:


file_name = "result_flux.txt"
save_flux_to_file(estimators, file_name)


# In[ ]:





 num1  1
i == 0
i == 1
i == 2
i == 3
i == 4
i == 5
i == 6
i == 7
i == 8
i == 9
i == 10
i == 11
i == 12
i == 13
i == 14
i == 15
i == 16
i == 17
i == 18
i == 19
i == 20
i == 21
i == 22
i == 23
i == 24
i == 25
i == 26
i == 27
i == 28
i == 29
i == 30
i == 31
i == 32
i == 33
i == 34
i == 35
i == 36
i == 37
i == 38
i == 39
i == 40
i == 41
i == 42
i == 43
i == 44
i == 45
i == 46
i == 47
i == 48
i == 49
i == 50
i == 51
i == 52
 keff_cycle , k_effective_exp, std_k_effective 2.693901464956837   2.6847809038165025  0.00912056114033466
i == 53
 keff_cycle , k_effective_exp, std_k_effective 2.6736951539425156   2.681085653858507  0.0064329685286235025
i == 54
 keff_cycle , k_effective_exp, std_k_effective 2.6642059783797283   2.6768657349888123  0.006204776975196997
i == 55
 keff_cycle , k_effective_exp, std_k_effective 2.6971461955304528   2.68092182709714  0.006288993367654529
i == 56
 keff_cycle , k_effective_exp, std_k_effective 2.662251309136327   2.6778100741036717  0.006004217829545392
i == 

 keff_cycle , k_effective_exp, std_k_effective 2.6906303473516537   2.6848444544979313  0.001904708026280017
i == 120
 keff_cycle , k_effective_exp, std_k_effective 2.7125466752516907   2.6852402005086993  0.0019185601170753394
i == 121
 keff_cycle , k_effective_exp, std_k_effective 2.681682395490414   2.6851900905788644  0.0018920088178160453
i == 122
 keff_cycle , k_effective_exp, std_k_effective 2.6673940017839857   2.6849429226789354  0.0018818483731388152
i == 123
 keff_cycle , k_effective_exp, std_k_effective 2.6485895504702377   2.6844449312788163  0.0019215424418965474
i == 124
 keff_cycle , k_effective_exp, std_k_effective 2.682463021505756   2.6844181487143155  0.0018955870026410116
i == 125
 keff_cycle , k_effective_exp, std_k_effective 2.656194194464542   2.6840418293243187  0.001907628466346035
i == 126
 keff_cycle , k_effective_exp, std_k_effective 2.684046704785873   2.6840418934751287  0.001882360749613789
i == 127
 keff_cycle , k_effective_exp, std_k_effective 2.718801

 keff_cycle , k_effective_exp, std_k_effective 2.647668886885995   2.683583259217775  0.0015033560809381557
i == 190
 keff_cycle , k_effective_exp, std_k_effective 2.7006513807374133   2.683705174371487  0.0014975499860476207
i == 191
 keff_cycle , k_effective_exp, std_k_effective 2.720024599222775   2.6839627589449  0.0015090377895615543
i == 192
 keff_cycle , k_effective_exp, std_k_effective 2.68772642524859   2.6839892636371796  0.001498607480067669
i == 193
 keff_cycle , k_effective_exp, std_k_effective 2.6681729749380443   2.683878660219703  0.0014921954845438365
i == 194
 keff_cycle , k_effective_exp, std_k_effective 2.6752313203905596   2.683818609248667  0.0014830130900908198
i == 195
 keff_cycle , k_effective_exp, std_k_effective 2.657742287660553   2.6836387725480595  0.0014836891547942346
i == 196
 keff_cycle , k_effective_exp, std_k_effective 2.696441140877046   2.6837264600023674  0.0014760986905224475
i == 197
 keff_cycle , k_effective_exp, std_k_effective 2.6801525136221

 keff_cycle , k_effective_exp, std_k_effective 2.658995554488125   2.683052462364518  0.0012122042100005152
i == 260
 keff_cycle , k_effective_exp, std_k_effective 2.6619100154200543   2.682951784045735  0.0012106116276609726
i == 261
 keff_cycle , k_effective_exp, std_k_effective 2.6978279548351276   2.6830222872248317  0.0012069214775093518
i == 262
 keff_cycle , k_effective_exp, std_k_effective 2.6996619053510083   2.683100775989578  0.0012037765028254214
i == 263
 keff_cycle , k_effective_exp, std_k_effective 2.6795632813259354   2.6830841680334108  0.001198226743149153
i == 264
 keff_cycle , k_effective_exp, std_k_effective 2.6926554414069273   2.6831288936099225  0.0011934527664993922
i == 265
 keff_cycle , k_effective_exp, std_k_effective 2.6794789399965424   2.6831119170814883  0.0011880101563471542
i == 266
 keff_cycle , k_effective_exp, std_k_effective 2.674703578388396   2.6830729895875387  0.001183137886270483
i == 267
 keff_cycle , k_effective_exp, std_k_effective 2.658005

 keff_cycle , k_effective_exp, std_k_effective 2.691083883866693   2.683571289708037  0.0010498128106750744
i == 330
 keff_cycle , k_effective_exp, std_k_effective 2.6970561840720393   2.683619450045051  0.0010471648213669223
i == 331
 keff_cycle , k_effective_exp, std_k_effective 2.6835133911089253   2.683619072611115  0.00104343166982779
i == 332
 keff_cycle , k_effective_exp, std_k_effective 2.6669992605484483   2.6835601371073468  0.0010413939765926592
i == 333
 keff_cycle , k_effective_exp, std_k_effective 2.6668586918100634   2.683501121399582  0.0010393844078568691
i == 334
 keff_cycle , k_effective_exp, std_k_effective 2.652664623411616   2.683392542181315  0.001041394021003945
i == 335
 keff_cycle , k_effective_exp, std_k_effective 2.6871487763051882   2.6834057219501704  0.0010378172654175433
i == 336
 keff_cycle , k_effective_exp, std_k_effective 2.6697508418549494   2.683357977614173  0.001035283669920757
i == 337
 keff_cycle , k_effective_exp, std_k_effective 2.69378781604

 keff_cycle , k_effective_exp, std_k_effective 2.6683679414299495   2.683102283268517  0.0009538053667805572
i == 400
 keff_cycle , k_effective_exp, std_k_effective 2.69540538130915   2.6831374349772044  0.0009517256849048579
i == 401
 keff_cycle , k_effective_exp, std_k_effective 2.719127619185565   2.68323997111455  0.0009545335452616448
i == 402
 keff_cycle , k_effective_exp, std_k_effective 2.668010295672984   2.6831967049911367  0.0009528007908768302
i == 403
 keff_cycle , k_effective_exp, std_k_effective 2.6957883502789777   2.6832323753743883  0.0009507671714400611
i == 404
 keff_cycle , k_effective_exp, std_k_effective 2.6854836189857725   2.6832387348196183  0.0009480989127071429
i == 405
 keff_cycle , k_effective_exp, std_k_effective 2.6951968748552035   2.6832724197211273  0.0009460243353790366
i == 406
 keff_cycle , k_effective_exp, std_k_effective 2.6875871476492255   2.683284539743397  0.0009434410746953822
i == 407
 keff_cycle , k_effective_exp, std_k_effective 2.6541787

 keff_cycle , k_effective_exp, std_k_effective 2.6797551012501755   2.6833590105595904  0.0008709303585031452
i == 470
 keff_cycle , k_effective_exp, std_k_effective 2.6806241969213045   2.683352499098547  0.0008688786393775399
i == 471
 keff_cycle , k_effective_exp, std_k_effective 2.680557147750694   2.683345859309122  0.0008668377677138759
i == 472
 keff_cycle , k_effective_exp, std_k_effective 2.6739694257158195   2.6833236402721714  0.0008650666027138759
i == 473
 keff_cycle , k_effective_exp, std_k_effective 2.6971568787500417   2.6833563429636085  0.0008636384900484881
i == 474
 keff_cycle , k_effective_exp, std_k_effective 2.6788854969433458   2.6833457985154476  0.0008616637194194583
i == 475
 keff_cycle , k_effective_exp, std_k_effective 2.684737347266194   2.683349072747802  0.000859640120054104
i == 476
 keff_cycle , k_effective_exp, std_k_effective 2.677145981657858   2.683334511501112  0.0008577434178745269
i == 477
 keff_cycle , k_effective_exp, std_k_effective 2.6661338